(numba_for_arviz)=
# Numba -  an overview

[Numba](https://numba.readthedocs.io/en/stable/index.html) is a just-in-time compiler for Python that works best on code that uses NumPy arrays and functions, and loops.
ArviZ includes {ref}`Numba as an optional dependency <Optional-dependencies>` and a number of functions have been included in `utils.py` for systems in which Numba is pre-installed. Additional functionality, {class}`arviz.Numba`, of disabling/re-enabling numba for systems that have Numba installed has also been included.

## A simple example to display the effectiveness of Numba

In [1]:
import arviz as az
import numpy as np
import timeit

from arviz.utils import conditional_jit, Numba
from arviz.stats.diagnostics import ks_summary

In [2]:
data = np.random.randn(1000000)

In [3]:
def variance(data, ddof=0):  # Method to calculate variance without using numba
    a_a, b_b = 0, 0
    for i in data:
        a_a = a_a + i
        b_b = b_b + i * i
    var = b_b / (len(data)) - ((a_a / (len(data))) ** 2)
    var = var * (len(data) / (len(data) - ddof))
    return var

In [4]:
%timeit variance(data, ddof=1)

140 ms ± 2.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
@conditional_jit
def variance_jit(data, ddof=0):  # Calculating variance with numba
    a_a, b_b = 0, 0
    for i in data:
        a_a = a_a + i
        b_b = b_b + i * i
    var = b_b / (len(data)) - ((a_a / (len(data))) ** 2)
    var = var * (len(data) / (len(data) - ddof))
    return var

In [6]:
%timeit variance_jit(data, ddof=1)

1.03 ms ± 44.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


That is almost 150 times faster!! Let's compare this to NumPy

In [7]:
%timeit np.var(data, ddof=1)

1.79 ms ± 124 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In certain scenarios, Numba can even outperform NumPy! 

## Numba within ArviZ
Let's see Numba's effect on a few of ArviZ functions

In [8]:
summary_data = np.random.randn(1000, 100, 10)
school = az.load_arviz_data("centered_eight").posterior["mu"].values

The methods of the {class}`~arviz.Numba` class can be used to enable or disable numba. The attribute `numba_flag` indicates whether numba is enabled within ArviZ or not.

In [9]:
Numba.disable_numba()
Numba.numba_flag

False

In [10]:
%timeit ks_summary(summary_data)

57.8 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit ks_summary(school)

462 µs ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
Numba.enable_numba()
Numba.numba_flag

True

In [13]:
%timeit ks_summary(summary_data)

7.18 ms ± 359 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit ks_summary(school)

359 µs ± 62.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Numba has provided a substantial speedup once again.